### LFS IRAQ

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyreadstat
import os

In [2]:
pd.set_option('display.max_columns', None)
os.chdir('C:/Users/511232/Desktop/LFS')
[f for f in os.listdir() if 'sav' in f]

['LFS_2021.sav']

In [3]:
#read in data and meta
df_lfs,meta_lfs=pyreadstat.read_sav('LFS_2021.sav', apply_value_formats=False)

col_names_lfs=meta_lfs.column_names_to_labels
col_vals_lfs=meta_lfs.variable_value_labels

In [4]:
df=df_lfs.copy()

#having_disability
'''
gen SSHD=0
replace SSHD = . if VIS_SS==. & HEAR_SS ==. & MOB_SS ==. & COM_SS==. & UB_SS==. & COG_SS==.
replace SSHD = 2 if VIS_SS==2 | HEAR_SS ==2 | MOB_SS ==2 | COM_SS==2 |UB_SS==2 | COG_SS==2
replace SSHD = 3 if VIS_SS==3 | HEAR_SS ==3 | MOB_SS ==3 | COM_SS==3 |UB_SS==3 | COG_SS==3
replace SSHD = 4 if VIS_SS==4 | HEAR_SS ==4 | MOB_SS ==4 | COM_SS==4 |UB_SS==4 | COG_SS==4
replace SSHD = 1 if SSHD ==0
drop if SSHD ==.
label define sshdlb 1 "None" 2 "Mild" 3 "Moderate" 4 "Severe"
label values SSHD sshdlb
'''

having_disability_labels={1.0: 'No, no difficulty',
 2.0: 'Yes, some difficulty',
 3.0: 'Yes, a lot of difficulty',
 4.0: 'Cannot do it at all',
 np.nan:'Not stated'}

dis_cols=['dif_sight','dif_hear','dif_mobi','dif_conc','dif_comm','dif_upper_body','dif_care']
df['having_disability']=df[dis_cols].apply(lambda x: x.max(), axis=1)
df['having_disability']=df['having_disability'].map(having_disability_labels)
#####################################################
#number of functional disability
''' 
{1.0: 'No, no difficulty',
 2.0: 'Yes, some difficulty',
 3.0: 'Yes, a lot of difficulty',
 4.0: 'Cannot do it at all'}
 
 if any of the disabilities have codes 1 or 2 they are not disabled, disabled otherwise
 -transform codes 3,4 to True and 1,2 to False and sum over axis=1'''

df['func_dis']=df[dis_cols].apply(lambda x: x.isin([3,4]), axis=1).sum(axis=1)
num_functional_dis_labels={0:'Zero domain', 1:'One domain', 2:'Two domains', 3:'Three or more domains'}
df['num_functional_dis']=np.select([df['func_dis']==1, df['func_dis']==2, df['func_dis']>=3], [1,2,3])
df['num_functional_dis']=df['num_functional_dis'].map(num_functional_dis_labels)
print('number of functional disability created')
#####################################################
#household type
'''
{1.0: 'Head',
 2.0: 'Spouse (Wife/Husband)',
 3.0: 'Child (Son/daughter)',
 4.0: 'Grandchild',
 5.0: 'Parent',
 6.0: 'Brother or Sister',
 7.0: 'Son or Daughter in law',
 8.0: 'Other relative',
 9.0: 'Non-relative',
 10.0: 'Domestic worker /Maid/Servant (Live-in)'}'''

hh_type_labels={1:'One person', 2:'Nuclear', 3:'Extended', 4:'Composite', 5: 'Unknown'}
#create a temporary var dem_rel_temp where na are replaced by 999
df['dem_rel_temp']=np.where(df['dem_rel'].isnull(), 999, df['dem_rel'])

def family_type(df):
    nuclear=[1,2,3,999]
    extended=nuclear+[4,5,6,7,8]
    composite=extended+[9,10]

    if len(df['dem_rel_temp'])==1:
        df['hh_type']=1
    elif all(df['dem_rel_temp'].isin(nuclear)):
        df['hh_type']=2
    elif all(df['dem_rel_temp'].isin(extended)):
        df['hh_type']=3
    elif all(df['dem_rel_temp'].isin(composite)):
        df['hh_type']=4
    else:
        df['hh_type']=5
    return(df)
#         
#create hh_type
#grouping by 2 variables will create a MultiIndex df
df=df.groupby(['cls_num','hh_num','area']).apply(family_type)

#create hh_type2: Population living alone in a household, Population living with family in a household
cond=[df['hh_type']==1, (df['hh_type']==2)|(df['hh_type']==3)|(df['hh_type']==4)]
rslt=['Population living alone in a household','Population living with family in a household']
df['hh_type2']=np.select(cond,rslt,default=np.nan)

#labeling hh_type
df['hh_type']=df['hh_type'].map(hh_type_labels)
print('hh_type and hh_type2 created')
#####################################################
#agegroup1: 0-4 5-9 10-14 15-19 20-24 25-29 30-34 35-39 40-44 45-49 50-54 55-59 60-64 65-69 70-74 75-7 80-84 85-89 90+

c1=(df['dem_ag1']>=0) & (df['dem_ag1']<=4)
c2=(df['dem_ag1']>=5) & (df['dem_ag1']<=9)
c3=(df['dem_ag1']>=10) & (df['dem_ag1']<=14)
c4=(df['dem_ag1']>=15) & (df['dem_ag1']<=19)
c5=(df['dem_ag1']>=20) & (df['dem_ag1']<=24)
c6=(df['dem_ag1']>=25) & (df['dem_ag1']<=29)
c7=(df['dem_ag1']>=30) & (df['dem_ag1']<=34)
c8=(df['dem_ag1']>=35) & (df['dem_ag1']<=39)
c9=(df['dem_ag1']>=40) & (df['dem_ag1']<=44)
c10=(df['dem_ag1']>=45) & (df['dem_ag1']<=49)
c11=(df['dem_ag1']>=50) & (df['dem_ag1']<=54)
c12=(df['dem_ag1']>=55) & (df['dem_ag1']<=59)
c13=(df['dem_ag1']>=60) & (df['dem_ag1']<=64)
c14=(df['dem_ag1']>=65) & (df['dem_ag1']<=69)
c15=(df['dem_ag1']>=70) & (df['dem_ag1']<=74)
c16=(df['dem_ag1']>=75) & (df['dem_ag1']<=79)
c17=(df['dem_ag1']>=80) & (df['dem_ag1']<=84)
c18=(df['dem_ag1']>=85) & (df['dem_ag1']<=89)
c19=(df['dem_ag1']>=90)

cond=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19]

rslt=['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
'40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84','85-89', '90+']

df['agegroup1']=np.select(cond,rslt,default=np.nan)
print('agegroup1 created')
#####################################################
#agegroup2: 18+
df['agegroup2']=np.select([df['dem_ag1']>=18, df['dem_ag1']<18],['18+','<18'],default=np.nan )
print('agegroup2 created')
#####################################################
#agegroup3: 5-14	15-24	25-34	35-44	45-54	55-64	65-74	75-84	85+
c1=(df['dem_ag1']>=5) & (df['dem_ag1']<=14)
c2=(df['dem_ag1']>=15) & (df['dem_ag1']<=24)
c3=(df['dem_ag1']>=25) & (df['dem_ag1']<=34)
c4=(df['dem_ag1']>=35) & (df['dem_ag1']<=44)
c5=(df['dem_ag1']>=45) & (df['dem_ag1']<=54)
c6=(df['dem_ag1']>=55) & (df['dem_ag1']<=64)
c7=(df['dem_ag1']>=65) & (df['dem_ag1']<=74)
c8=(df['dem_ag1']>=75) & (df['dem_ag1']<=84)
c9=df['dem_ag1']>=85

cond=[c1,c2,c3,c4,c5,c6,c7,c8,c9]
rslt=['5-14','15-24','25-34','35-44','45-54','55-64','65-74','75-84','85+']
df['agegroup3']=np.select(cond,rslt,default=np.nan)
print('agegroup3 created')
#####################################################
# agegroup4: <15	15-19	20-24	25-29	30-34	35-39	40-44	45-49	50-54	55-59	60-64	65+
c1=df['dem_ag1']<15
c2=(df['dem_ag1']>=15) & (df['dem_ag1']<=19)
c3=(df['dem_ag1']>=20) & (df['dem_ag1']<=24)
c4=(df['dem_ag1']>=25) & (df['dem_ag1']<=29)
c5=(df['dem_ag1']>=30) & (df['dem_ag1']<=34)
c6=(df['dem_ag1']>=35) & (df['dem_ag1']<=39)
c7=(df['dem_ag1']>=40) & (df['dem_ag1']<=44)
c8=(df['dem_ag1']>=45) & (df['dem_ag1']<=49)
c9=(df['dem_ag1']>=50) & (df['dem_ag1']<=54)
c10=(df['dem_ag1']>=55) & (df['dem_ag1']<=59)
c11=(df['dem_ag1']>=60) & (df['dem_ag1']<=64)
c12=df['dem_ag1']>=65

cond=[c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12]
rslt=['<15','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65+']
df['agegroup4']=np.select(cond,rslt,default=np.nan)
print('agegroup4 created')
#####################################################
#agegroup5 <18	18-24	25-64	65+
c1=df['dem_ag1']<18
c2=(df['dem_ag1']>=18) & (df['dem_ag1']<=24)
c3=(df['dem_ag1']>=25) & (df['dem_ag1']<=64)
c4=df['dem_ag1']>=65

cond=[c1,c2,c3,c4]
rslt=['<18','18-24','25-64','65+']
df['agegroup5']=np.select(cond,rslt,default=np.nan)
print('agegroup5 created')
#####################################################
#age5+, age15+, age25+
df['age5+']=np.where(df['dem_ag1']>=5, '5+', '<5')
df['age15+']=np.where(df['dem_ag1']>=15, '15+', '<15')
df['age25+']=np.where(df['dem_ag1']>=25, '25+', '<25')
#####################################################
#agegroup6 15-29	30-44	45-64	65+
c1=(df['dem_ag1']>=15) & (df['dem_ag1']<=29)
c2=(df['dem_ag1']>=30) & (df['dem_ag1']<=44)
c3=(df['dem_ag1']>=45) & (df['dem_ag1']<=64)
c4=df['dem_ag1']>=65

cond=[c1,c2,c3,c4]
rslt=['15-29','30-44','45-64','65+']
df['agegroup6']=np.select(cond,rslt,default=np.nan)
print('agegroup6 created')

#####################################################
#agegroup7: 5-9 10-14 15-19 20-24 25-29 30+
c1=(df['dem_ag1']>=5) & (df['dem_ag1']<=9)
c2=(df['dem_ag1']>=10) & (df['dem_ag1']<=14)
c3=(df['dem_ag1']>=15) & (df['dem_ag1']<=19)
c4=(df['dem_ag1']>=20) & (df['dem_ag1']<=24)
c5=(df['dem_ag1']>=25) & (df['dem_ag1']<=29)
c6=df['dem_ag1']>=30

cond=[c1,c2,c3,c4,c5,c6]
rslt=['5-9', '10-14', '15-19', '20-24', '25-29', '30+']

df['agegroup7']=np.select(cond,rslt,default=np.nan)
print('agegroup7 created')
#####################################################
#agegroup8: 15-24	25-64	65+
c1=(df['dem_ag1']>=15) & (df['dem_ag1']<=24)
c2=(df['dem_ag1']>=25) & (df['dem_ag1']<=63)
c3=df['dem_ag1']>=64

cond=[c1,c2,c3]
rslt=['15-24', '25-64', '64+']

df['agegroup8']=np.select(cond,rslt,default=np.nan)
print('agegroup8 created')
#####################################################
#relationship to HH
c1=df['dem_rel']==1
c2=df['dem_rel'].isin([2,3,4,5,6,7,8,9,10])
df['hh_relationship']=np.select([c1,c2],[1,2],default=999)
df['hh_relationship']=df['hh_relationship'].map({1:'head of HH', 2:'Other', 999:'Not stated'})
print('hh_relationship created')
#####################################################
#household size: 1	2	3	4	5	6	7	8+
df['hh_size']=np.where(df['ghhhsize']>=8, '8+',df['ghhhsize'])
print('hh_size created')
#####################################################
#recode edu_attain
cond1=(df['dem_edl']==1) | (df['dem_edl']==2) | (df['dem_edl']==3)
cond2=df['dem_edl']==4
cond3=(df['dem_edl']==5) | (df['dem_edl']==6)
cond4=(df['dem_edl']==8) | (df['dem_edl']==9)
cond5=(df['dem_edl']==10)
cond6=(df['dem_edl']==11) | (df['dem_edl']==12)
cond7=(df['dem_edl']==13)
cond8=(df['dem_edl']==14)
cond9=(df['dem_edl']==7) | (df['dem_edl']==15)
cond10=df['dem_edl'].isnull()

cond=[cond1,cond2,cond3,cond4,cond5,cond6,cond7,cond8,cond9,cond10]
rslt=[1,2,3,4,5,6,7,8,9,99]

df['edu_attain']=np.select(cond,rslt)
print('edu_attain created')
#####################################################
#recode edu_attend
df['edu_attend']=df['dem_edc'].fillna(999)
df['edu_attend'].map({1:"Attending", 2:"Not attending", 99:"Not stated"})
print('edu_attend created')
#####################################################
#recode edu_level
cond1=df['edu_attain']==1
cond2=df['edu_attain']==2
cond3=(df['edu_attain']==3) | (df['edu_attain']==4) | (df['edu_attain']==5)
cond4=(df['edu_attain']==6) | (df['edu_attain']==7) | (df['edu_attain']==8)
cond5=(df['edu_attain']==9)
cond6=df['edu_attain'].isnull()

cond=[cond1,cond2,cond3,cond4,cond5,cond6]
rslt=[1,2,3,4,5,99]

#map lables for both edu_attain and edu_level
education_label={1:"No Schooling",2:"Primary Education",3:"Secondary Education",4:"Tertiary Education",5:"Other",99:"Not stated" }
df['edu_level']=np.select(cond,rslt)
df['edu_level'].map(education_label)

#edu_attain labels
edu_attain_labels={1:"ISCED Level X: No schooling", 2:"ISCED Level 1: Primary education", 3:"ISCED Level 2: Lower secondary education", 4:"ISCED level 3: Upper secondary education", 5:"ISCED Level 4 : Post-secondary education non-tertiary", 6:"ISCED Level 6: Bachelor's or equivalent level", 7:"ISCED level 7: Master's or equivalent level", 8:"ISCED level 8: Doctoral or equivalent level",9:"Not classifiable by level and grade of education", 99:"Not stated"}

df['edu_attain']=df['edu_attain'].map(edu_attain_labels)
print('edu_level created')
#####################################################
#recode literacy






#####################################################
#add labels
vars=['dem_sex','area','dif_sight','dif_hear','dif_mobi','dif_conc','dif_care','dif_comm','dif_upper_body','dif_sight_reason','dif_hear_reason','dif_mobi_reason','dif_comm_reason','dif_conc_reason','dif_care_reason','dif_upper_body_reason','dem_mrt','ilo_edu_isced11_Yves']

for v in vars:
    df[v]=df[v].map(col_vals_lfs[v])

number of functional disability created
hh_type and hh_type2 created
agegroup1 created
agegroup2 created
agegroup3 created
agegroup4 created
agegroup5 created
agegroup6 created
agegroup7 created
agegroup8 created
hh_relationship created
hh_size created
edu_attain created
edu_attend created
edu_level created


agegroup7 created


## CROSSTABS

In [ ]:
#to supress scientific notation
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
#Table 1 xtab_all_dis_ByAge###########################
newcol=['0-4','5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
       '40-44', '45-49','50-54', '55-59', '60-64', '65-69', '70-74',
       '75-79', '80-84', '85-89', '90+', '18+', '<18']

xtab_total=[]
for col in ['agegroup1', 'agegroup2']:

       xtab=pd.crosstab([df['area'],df['having_disability'],df['dem_sex']],df[col],
       rownames=['Area','Disability Type', 'Sex'],
       colnames=['agegroup'], 
       values=df['weight'], aggfunc='sum',dropna=False)
       #drop the column 'na' from the dataframe
       xtab.drop(['nan'], axis=1, inplace=True)
       xtab_total.append(xtab)

#append the dataframes
T=pd.concat(xtab_total, axis=1) 

#reindex the columns to rearrange them
T=T.reindex(newcol, axis='columns')
  
T.to_excel('Table 1 xtab_all_dis_ByAge.xlsx')
print('Table 1: population_age generated and saved')

In [ ]:
#Table 2 DisabType_Age###########################
dis_cols=['dif_sight','dif_hear','dif_mobi','dif_comm','dif_conc','dif_care','dif_upper_body']
newcol=['5-14','15-24','25-34','35-44','45-54','55-64','65-74','75-84','85+']

xtab_total=[]
for dis in dis_cols:
    #filter dataframe
    df1=df[df[dis].isin(['Yes, a lot of difficulty','Cannot do it at all'])].copy()
    xtab=pd.crosstab([df1['area'],df1[dis],df1['dem_sex']],df1['agegroup3'],
    rownames=['Area','Disability Type', 'Sex'],
    colnames=['agegroup'], 
    values=df['weight'], aggfunc='sum',dropna=False)

    xtab=xtab.reindex(newcol, axis='columns')
    #add the name of the disability as an index
    # T['disability'] = dis
    # T.set_index('disability', append=True, inplace=True)

    #add the diability name as a new level on the columns
    #replicate the disability name as much as the column length
    #zip to create tuples and then use them as the new columns
    head1=[dis]*len(xtab.columns)
    xtab.columns=tuple(zip(head1,xtab.columns))
    xtab_total.append(xtab)
            
T=pd.concat(xtab_total, axis=1)

T.to_excel('Table 2 DisabType_Age.xlsx')
print('Table 2: DisabType_Age generated and saved')

In [ ]:
#Table 3 CauseType###########################
dis_reasons=['dif_sight_reason','dif_hear_reason','dif_mobi_reason','dif_comm_reason','dif_conc_reason','dif_care_reason','dif_upper_body_reason']

xtab_total=[]
for r in dis_reasons:
    #map labels
    xtab=pd.crosstab([df['area'],df['dem_sex']],df[r],
    rownames=['Area','Sex'],
    colnames=[r],
    values=df['weight'], aggfunc='sum',dropna=False)
    #add column name as multiindex column
    head1=[r]*len(xtab.columns)
    xtab.columns=tuple(zip(head1,xtab.columns))
    xtab_total.append(xtab)
            
T=pd.concat(xtab_total, axis=1)
T.to_excel('Table 3 CauseType.xlsx')
print('Table 3: CauseType generated and saved')

In [ ]:
#Table 4 Number_DisabilityDomain###########################
xtab=pd.crosstab([df['area'],df['having_disability'],df['dem_sex']],df['num_functional_dis'],
rownames=['Area','Disability Type', 'Sex'],
colnames=['num_functional_dis'], 
values=df['weight'], aggfunc='sum',dropna=False)

xtab.to_excel('Table 4 Number_DisabilityDomain.xlsx')
print('Table 4: Number_DisabilityDomain generated and saved')


In [ ]:
#Table 5 MaritalStatus###########################
xtab=pd.crosstab([df['area'],df['dem_mrt'],df['having_disability'],df['dem_sex']],df['agegroup4'],
rownames=['Area','marital status','Disability type','Sex'],
colnames=['Agegroup'],
values=df['weight'], aggfunc='sum',dropna=False)

xtab.to_excel('Table 5 MaritalStatus.xlsx')
print('Table 5: MaritalStatus generated and saved')


In [ ]:
#Table 6 Head_HH###########################
xtab=pd.crosstab([df['area'],df['having_disability'],df['dem_sex']],df['hh_relationship'],
rownames=['Area','Disability type','Sex'],
colnames=['relationship to HH'],
values=df['weight'], aggfunc='sum',dropna=False)

xtab.to_excel('Table 6 Head_HH.xlsx')

print('Table 6: Head_HH generated and saved')

In [33]:
#Table 7 HH_Type&Size###########################
#for age >=18
df1=df[df['dem_ag1']>=18].copy()

df_total=[]
for col in ['having_disability','hh_size']:

    xtab=pd.crosstab([df1['area'],df1['hh_type']],df1[col],
    rownames=['Area','HH type'],
    colnames=[col],
    values=df1['weight'], aggfunc='sum',dropna=False)

    df_total.append(xtab)

T=pd.concat(df_total, axis=1)
T.to_excel('Table 7 HH_Type&Size.xlsx')
print('Table 7: HH_Type&Size generated and saved')

Table 7: HH_Type&Size generated and saved


In [24]:
#Table 8: Living_Type_Age###########################
dis_cols=['dif_sight','dif_hear','dif_mobi','dif_comm','dif_conc','dif_care','dif_upper_body']
newcol=['<18','18-24','25-64','65+']

xtab_total=[]
for dis in dis_cols:
    #filter dataframe
    df1=df[df[dis].isin(['Yes, a lot of difficulty','Cannot do it at all'])].copy()
    xtab=pd.crosstab([df1['area'],df1['hh_type2'],df1[dis],df1['dem_sex']],df1['agegroup5'],
    rownames=['Area','hh type','Disability Type', 'Sex'],
    colnames=['agegroup'], 
    values=df['weight'], aggfunc='sum',dropna=False)

    xtab=xtab.reindex(newcol, axis='columns')
    #add the name of the disability as an index
    # T['disability'] = dis
    # T.set_index('disability', append=True, inplace=True)

    #add the diability name as a new level on the columns
    #replicate the disability name as much as the column length
    #zip to create tuples and then use them as the new columns
    head1=[dis]*len(xtab.columns)
    xtab.columns=tuple(zip(head1,xtab.columns))
    xtab_total.append(xtab)
            
T=pd.concat(xtab_total, axis=1)

T.to_excel('Table 8 Living_Type_Age.xlsx')
print('Table 8: Living_Type_Age generated and saved')

Table 8: Living_Type_Age generated and saved


In [6]:
#Table 9: Living_Type_Age (new)###########################
dis_cols=['dif_sight','dif_hear','dif_mobi','dif_comm','dif_conc','dif_care','dif_upper_body']
newcol=['15-24','25-34','35-44','45-54','55-64','65-74','75-84','85+']

#filter dataframe on age
df1=df[df['dem_ag1']>=15].copy()
xtab_total=[]
for dis in dis_cols:
    #filter dataframe
    df2=df1[df1[dis].isin(['Yes, a lot of difficulty','Cannot do it at all'])].copy()
    xtab=pd.crosstab([df1['area'],df1['hh_type2'],df1[dis],df1['dem_sex']],df1['agegroup3'],
    rownames=['Area','hh type','Disability Type', 'Sex'],
    colnames=['agegroup'], 
    values=df['weight'], aggfunc='sum',dropna=False)

    xtab=xtab.reindex(newcol, axis='columns')
    #add the name of the disability as an index
    # T['disability'] = dis
    # T.set_index('disability', append=True, inplace=True)

    #add the diability name as a new level on the columns
    #replicate the disability name as much as the column length
    #zip to create tuples and then use them as the new columns
    head1=[dis]*len(xtab.columns)
    xtab.columns=tuple(zip(head1,xtab.columns))
    xtab_total.append(xtab)
            
T=pd.concat(xtab_total, axis=1)

T.to_excel('Table 9 Living_Type_Age(new).xlsx')
print('Table 9: Living_Type_Age(new) generated and saved')

Table 9: Living_Type_Age(new) generated and saved


In [39]:
#Table 10 EducatAttaintment_Age###########################
cols=[('age5+',"df['dem_ag1']>=5"),('age15+',"df['dem_ag1']>=15"),('age25+',"df['dem_ag1']>=25"),('agegroup6',"df['dem_ag1']>=15")]
df_total=[]

for c in cols:
    #filter the dataframe
    df1=df[eval(c[1])].copy()
    xtab=pd.crosstab([df1['area'],df1['having_disability'],df1['dem_sex']],[df1['edu_attain'],df1[c[0]]],
    rownames=['Area','Disability','Sex'],
    colnames=['edu','age'],
    values=df['weight'], aggfunc='sum',dropna=False)
    df_total.append(xtab)

T=pd.concat(df_total, axis=1)
T.to_excel('Table 10 EducatAttaintment_Age.xlsx')
print('Table 10: EducatAttaintment_Age generated and saved')

Table 10: EducatAttaintment_Age generated and saved


In [24]:
#Table 11: EducatAttainment_Type###########################
dis_cols=['dif_sight','dif_hear','dif_mobi','dif_comm','dif_conc','dif_care','dif_upper_body']
cols=[('age5+',"df['dem_ag1']>=5"),('age15+',"df['dem_ag1']>=15"),('age25+',"df['dem_ag1']>=25"),('agegroup6',"df['dem_ag1']>=15")]

df_list_ages=[]
df_Total=[]
for dis in dis_cols:
    for c in cols:
        #filter the dataframe
        df1=df[eval(c[1])].copy()
        xtab=pd.crosstab([df1['area'],df1[dis],df1['dem_sex']],[df1['edu_attain'],df1[c[0]]],
        rownames=['Area','Disability','Sex'],
        colnames=['edu','age'],
        values=df['weight'], aggfunc='sum',dropna=False)
        #add the dis as a row index
        xtab=pd.concat([xtab], keys=[dis], axis=0)
        df_list_ages.append(xtab)

    #concat the generated tables for all ages
    T=pd.concat(df_list_ages, axis=0)
    #append to df_Total
    df_Total.append(T)

#concat df_Total on axis=0
T_final=pd.concat(df_Total, axis=0)
T_final.to_excel('Table 11 EducatAttainment_Type.xlsx')
print('Table 11: EducatAttainment_Type generated and saved')

Table 11: EducatAttainment_Type generated and saved


In [49]:
#Table 12: School_attendance###########################
#filter the dataframe
df1=df[df['dem_ag1']>=5].copy()

newcol=['5-9', '10-14', '15-19', '20-24', '25-29', '30+']
xtab=pd.crosstab([df1['area'],df1['edu_attend'],df1['having_disability'],df1['dem_sex']],[df1['agegroup7']],
rownames=['Area','edu_attendance','disability', 'sex'],
colnames=['age'],
values=df['weight'], aggfunc='sum',dropna=False)

xtab=xtab.reindex(newcol, axis='columns')
xtab.to_excel('Table 12 School_attendance.xlsx')

print('Table 12: School_attendance generated and saved')

Table 12: School_attendance generated and saved


In [ ]:
#Table 13: SchoolAttendance_Type###########################


print('Table 13: SchoolAttendance_Type generated and saved')

In [ ]:
# #Table 14: Literacy###########################
# #filter the dataframe
# df1=df[df['dem_ag1']>=15].copy()

# newcol=['15-24', '25-64', '65+']
# xtab=pd.crosstab([df1['area'],df1['having_disability'],df1['dem_sex']],[df1['literacy'],df1['agegroup8']],
# rownames=['Area','disability', 'sex'],
# colnames=['literacy','age'],
# values=df['weight'], aggfunc='sum',dropna=False)

# xtab=xtab.reindex(newcol, axis='columns')
# xtab.to_excel('Table 14 14 Literacy.xlsx')

# print('Table 14: Literacy generated and saved')

In [ ]:
#Table 15: Literacy_Type_Age###########################


print('Table 15: Literacy_Type_Age generated and saved')

In [ ]:
#Table 16: ActivityStatus###########################


print('Table 16: ActivityStatus generated and saved')

In [ ]:
#Table 17: ActivityStatus_Education###########################


print('Table 17: ActivityStatus_Education generated and saved')

In [ ]:
#Table 18: ActivityStatus_Type###########################


print('Table 18: ActivityStatus_Type generated and saved')

In [ ]:
#Table 19: LaborForce###########################


print('Table 19: LaborForce generated and saved')

In [ ]:
#Table 20: LaborForce_Type###########################


print('Table 20: LaborForce_Type generated and saved')

In [ ]:
#Table 21: Reason_Not_Seeking_Job###########################


print('Table 21: Reason_Not_Seeking_Job generated and saved')

In [ ]:
#Table 22: Youth_NEET generated###########################


print('Table 22: Youth_NEET generated and saved')

In [ ]:
#Table 23: Youth_NEET_Type###########################


print('Table 23: Youth_NEET_Type generated and saved')

In [ ]:
#Table 24: EducatAttaintment_Age###########################


print('Table 24: EducatAttaintment_Age generated and saved')

In [ ]:
#Table 25: Occupation###########################


print('Table 25: Occupation generated and saved')

In [ ]:
#Table 26: Sector###########################


print('Table 26: Sector generated and saved')

In [ ]:
#Table 27: Sector_type###########################


print('Table 27: Sector_type generated and saved')

In [ ]:
#Table 28: Type_of_ownership###########################


print('Table 28: Type_of_ownership generated and saved')

In [ ]:
#Table 29: Grants_Benefits_Insurance###########################


print('Table 29: Grants_Benefits_Insurance generated and saved')